In [1]:
import pandas as pd

from collections import Counter

import itertools

In [2]:
import glob

import numpy as np

import tqdm

In [3]:
import os

os.chdir('..')


from importlib.machinery import SourceFileLoader

equilibria = SourceFileLoader("eq", "src/equilibria.py").load_module()

main = SourceFileLoader("main", "src/main.py").load_module()

In [4]:
import random

In [5]:
def seperate_equilibria_and_non(df):
    per_player = df.groupby('ID')['condition A'].all()

    eq_players_ID = per_player[per_player == True].index

    noneq_players_ID = per_player[per_player == False].index
    
    return df[df['ID'].isin(eq_players_ID)], df[df['ID'].isin(noneq_players_ID)]

**Two bit against memory-two**

In [6]:
columns = (["", "ID"] + [f'p{i+1}' for i in range(16)] + [f'q{i+1}' for i in range(16)] + 
           ['label', 'Sp', 'Sq', "condition A", "condition B",'b', 'c'])

files = glob.glob("two_bit_in_memory_two/*csv")

In [7]:
to_read = random.choices(files, k=100)

In [8]:
dfs = [pd.read_csv(file, index_col=0, names=columns) for file in to_read]

In [9]:
reactive = pd.concat(dfs)

In [10]:
def conditition_one(p2, p4, b, c):
    return p4 < 1 - (c / b)

def conditition_two(p2, p4, b, c):
    return p2 < p4

def conditition_three(p3, b, c):
    return p3 < 1

def condition_four(p2, p4, b, c):
    return  1 + p2 < (b / c) + (p4 * (c - b)) / c

def nash_conditions(p, b, c):
    p1, p2, p3, p4 = p
    
    c1 = conditition_one(p2, p4, b, c)
    c2 = conditition_two(p2, p4, b, c)
    c3 = conditition_three(p3, b, c)
    c4 = condition_four(p2, p4, b, c)
    
    return {"c1": c1, "c2": c2, "c3": c3, "c4": c4}

In [11]:
eq, noneq = seperate_equilibria_and_non(reactive)

In [12]:
for i, row in noneq[['p1', 'p2', 'p3', 'p4']].drop_duplicates().iterrows():
    p = row[['p1', 'p2', 'p3', 'p4']]
    
    check = nash_conditions(p, 2, 1)
    
    assert all(check.values()) == False

**Memory-two against memory-two**

In [6]:
def invariant_distributions(M):
    """Returns all the invariant distributions in case of absorbing states."""
    
    stationaries = []
    
    eigenvalues, eigenvectors = np.linalg.eig(M.T)
    
    for index in np.where(eigenvalues == np.max(eigenvalues))[0]:
        
        eigenvectors_one = eigenvectors[:, index]

        stationary = eigenvectors_one / eigenvectors_one.sum()

        stationaries.append(stationary.real)
        
    return stationaries

In [7]:
def conditions(p, b, c):
    c1  = p[1] < p[5]
    c2  = p[9] < p[5]
    c3  = p[13] < p[5]
    c4  = p[4] < 1
    c5  = p[8] < 1
    c6  = p[12] < 1
    c7  = (c / b) * p[2] < 1 - p[5]
    c8  = (c / b) * p[10] < 1 - p[5]
    c9  = (c / b) * p[14] < 1 - p[5]
    c12 = (c / b) * p[6] < 1 - p[5]
    c10 = 1 + (c / (b - c)) * p[11] > p[5]
    c11 = 1 + (c / (b - c)) * p[15] > p[5]
    c13 = (c / (b - c)) * p[7] < 1 - p[5]
    c14 = 1 + (c / (b - c)) * p[3] > p[5]
    
    return  {
"c1" : c1 ,
"c2" : c2 ,
"c3" : c3 ,
"c4" : c4 ,
"c5" : c5 ,
"c6" : c6 ,
"c7" : c7 ,
"c8" : c8 ,
"c9" : c9 ,
"c12" : c12,
"c10" : c10,
"c11" : c11,
"c13" : c13,
"c14" : c14,
}
    

In [8]:
np.random.seed(0)

In [9]:
nash_strategies = []

for i in range(200):
    strategy = np.random.random((1, 16)).round(5)[0]
    strategy[0] = 1
    
    if all(conditions(strategy, 2, 1).values()):
        nash_strategies.append(strategy)
        

In [10]:
nash_strategies

[array([1.     , 0.1033 , 0.86717, 0.02919, 0.53492, 0.40424, 0.52418,
        0.3651 , 0.19057, 0.01912, 0.51815, 0.84278, 0.37322, 0.22286,
        0.08053, 0.08531]),
 array([1.     , 0.0666 , 0.23597, 0.1531 , 0.19752, 0.52832, 0.67169,
        0.47032, 0.9597 , 0.24029, 0.76314, 0.87018, 0.56207, 0.45622,
        0.59618, 0.42881]),
 array([1.     , 0.11279, 0.99045, 0.84537, 0.53451, 0.42455, 0.28646,
        0.50159, 0.87942, 0.27501, 0.50054, 0.23455, 0.33715, 0.19026,
        0.99054, 0.5715 ]),
 array([1.     , 0.22214, 0.87972, 0.15576, 0.28059, 0.38614, 0.27364,
        0.21071, 0.77115, 0.32902, 0.11251, 0.5409 , 0.12549, 0.35752,
        0.88183, 0.36916])]

In [28]:
def run(i, strategy, coplayers, labels, filename, Sx, b, c):

    sx = match_payoff(strategy, strategy, Sx)
    data = []

    for label, coplayer in tqdm.notebook.tqdm(zip(labels, coplayers)):

        sy = match_payoff(coplayer, strategy, Sx)
        A = np.isclose(sx, sy, atol=10 ** -4) or sx > sy
        B = np.isclose(sy, b - c, atol=10 ** -4) or sy < b - c


        data_point = [
            i,
            label,
            sx,
            sy,
            A,
            B,
            b,
            c,
        ]
        data.append(data_point)
        
    return data

In [29]:
def match_payoff(player, coplayer, Sx):
    M = equilibria.calculate_M_memory_two(player, coplayer)
    ss = main.invariant_distribution(M)

    return ss @ Sx

In [30]:
b, c, dimensions = 2, 1, 2

deterministic_strategies = list(
    itertools.product([0, 1], repeat=2 ** (2 * dimensions))
)

labels = [f"N{i}" for i, _ in enumerate(deterministic_strategies)]

Sx = equilibria.payoffs_donation(b, c, dim=(2 * dimensions))  # payoffs(R, P, dim=4)

In [31]:
len(labels), len(deterministic_strategies)

(65536, 65536)

In [34]:
strategy = nash_strategies[0]

coplayers = deterministic_strategies

sx = match_payoff(strategy, strategy, Sx)
data = []

for label, coplayer in tqdm.tqdm_notebook(zip(labels, coplayers)):

    sy = match_payoff(coplayer, strategy, Sx)
    A = np.isclose(sx, sy, atol=10 ** -4) or sx > sy
    B = np.isclose(sy, b - c, atol=10 ** -4) or sy < b - c


    data_point = [
        i,
        label,
        sx,
        sy,
        A,
        B,
        b,
        c,
    ]
    data.append(data_point)

<ipython-input-34-999ef95dc697>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for label, coplayer in tqdm.tqdm_notebook(zip(labels, coplayers)):


In [35]:
all([d[4] for d in data])

False

In [37]:
[d[1] for d in data if d[4] == False]

['N39616', 'N43840', 'N58244', 'N61828', 'N61892', 'N63048', 'N63940']

In [41]:
data = run(
        1,
        nash_strategies[0],
        deterministic_strategies,
        labels,
        "",
        Sx,
        b,
        c,
    )

In [42]:
all([d[4] for d in data])

False

In [43]:
[d[1] for d in data if d[4] == False]

['N39616', 'N43840', 'N58244', 'N61828', 'N61892', 'N63048', 'N63940']

In [75]:
M = equilibria.calculate_M_memory_two(deterministic_strategies[63940], strategy)

ss = main.invariant_distribution(M)

In [76]:
ss.round(4) #@ np.array([1, -1, 2, 0] * 4)

array([ 1.7526, -0.    , -0.    ,  0.    , -0.    , -0.    , -0.    ,
       -0.    , -0.    , -0.    , -0.0281, -0.1441, -0.    , -0.    ,
       -0.1441, -0.4364])

In [77]:
eigenvalues, eigenvectors = np.linalg.eig(M.T)

In [87]:
np.where(np.isclose(eigenvalues, 1))[0]

array([0, 6])

In [90]:
all(stationary > 0)

False

In [134]:

def invariant_distribution_for_mem_two(M):
    stationaries = []

    eigenvalues, eigenvectors = np.linalg.eig(M.T)

    for index in np.where(np.isclose(eigenvalues, 1))[0]:

        eigenvectors_one = eigenvectors[:, index]

        stationary = eigenvectors_one / eigenvectors_one.sum()


        stationaries.append(stationary.real)

    return stationaries[np.argmax([min(s) for s in stationaries])]


In [135]:
def match_payoff(player, coplayer, Sx):
    M = equilibria.calculate_M_memory_two(player, coplayer)
    ss = invariant_distribution_for_mem_two(M)

    return ss @ Sx

In [136]:
data = run(
        1,
        nash_strategies[1],
        deterministic_strategies,
        labels,
        "",
        Sx,
        b,
        c,
    )

In [115]:
[d[1] for d in data if d[4] == False]

[]

In [137]:
strategy = [1., 0.56843, 0.01879, 0.61764, 0.6121,  0.61693, 0.94375, 0.68182, 0.35951,
 0.43703, 0.69763, 0.06023, 0.66677, 0.67064, 0.21038, 0.12893]

In [138]:
data = run(
        1,
        strategy,
        deterministic_strategies,
        labels,
        "",
        Sx,
        b,
        c,
    )

In [139]:
M = equilibria.calculate_M_memory_two(deterministic_strategies[29452], strategy)

In [140]:
np.sum(M, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [11]:
columns = (["", "ID"] + [f'p{i+1}' for i in range(16)] + [f'q{i+1}' for i in range(16)] + 
           ['label', 'Sp', 'Sq', "condition A", "condition B",'c', 'b'])

files = glob.glob("memory_two_local/*csv")
    
dfs = [pd.read_csv(file, index_col=0, names=columns) for file in files]

In [12]:
local = pd.concat(dfs)

In [13]:
len(files)

300

In [14]:
eq, noneq = seperate_equilibria_and_non(local)

In [15]:
local = local.reset_index(drop=True)

In [16]:
eq.head()

,ID,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,q14,q15,q16,label,Sp,Sq,condition A,condition B,c,b
,,,,,,,,,,,,,,,,,,,,,
0,447,1.0,0.48925,0.70679,0.5452,0.3168,0.15254,0.5406,0.14216,0.9393,...,0,0,0,N0,1.0,0.407494,True,True,2,1
1,447,1.0,0.48925,0.70679,0.5452,0.3168,0.15254,0.5406,0.14216,0.9393,...,0,0,1,N1,1.0,0.508966,True,True,2,1
2,447,1.0,0.48925,0.70679,0.5452,0.3168,0.15254,0.5406,0.14216,0.9393,...,0,1,0,N2,1.0,0.465121,True,True,2,1
3,447,1.0,0.48925,0.70679,0.5452,0.3168,0.15254,0.5406,0.14216,0.9393,...,0,1,1,N3,1.0,0.545173,True,True,2,1
4,447,1.0,0.48925,0.70679,0.5452,0.3168,0.15254,0.5406,0.14216,0.9393,...,1,0,0,N4,1.0,0.407494,True,True,2,1


In [19]:
players = noneq[[f'p{i+1}' for i in range(16)]].drop_duplicates()

In [20]:
# players = local[[f'p{i+1}' for i in range(16)]].drop_duplicates()

In [21]:
players = players.reset_index(drop=True)

In [22]:
for i, row in players.iterrows():
    if all(conditions(row.values, 2, 1).values()):
        print(i)

In [23]:
players = eq[[f'p{i+1}' for i in range(16)]].drop_duplicates()
players = players.reset_index(drop=True)

In [24]:
conditions(players.iloc[0].values, 2, 1)

{'c1': False,
 'c2': False,
 'c3': False,
 'c4': True,
 'c5': True,
 'c6': True,
 'c7': True,
 'c8': True,
 'c9': True,
 'c12': True,
 'c10': True,
 'c11': True,
 'c13': True,
 'c14': True}

In [160]:
players.iloc[0]

p1     1.00000
p2     0.48925
p3     0.70679
p4     0.54520
p5     0.31680
p6     0.15254
p7     0.54060
p8     0.14216
p9     0.93930
p10    0.53989
p11    0.17492
p12    0.72064
p13    0.10355
p14    0.77164
p15    0.45729
p16    0.13266
Name: 0, dtype: float64